# Rational algebra example

By William Davis

## Motivation

When solving linear algebra problems, it can be useful to use a computer to automate the calculations. However, Python's implementation of matrix operations often reverts to floating point representation of data. These incur a  small but noticeable error. 

As an example, say I have a $5\times 5$ matrix which I want to find the inverse of.

In [1]:
import numpy as np

In [2]:
def matrixMultiplier(C, N):
    M = np.zeros((N, N))
    M = M.astype(int)
    exponentRange = range(N)
    for i in exponentRange:
        M[:,i] = [C[i]**x for x in exponentRange]
    return M

In [3]:
intM = matrixMultiplier([-2,-1,0,1,2], 5)
print(intM)

[[ 1  1  1  1  1]
 [-2 -1  0  1  2]
 [ 4  1  0  1  4]
 [-8 -1  0  1  8]
 [16  1  0  1 16]]


If we use `numpy`'s linear algebra inverse function, the result is a matrix of floating point numbers with small errors.

In [4]:
intMinv = np.linalg.inv(intM)
print(intMinv)

[[-1.00228468e-18  8.33333333e-02 -4.16666667e-02 -8.33333333e-02
   4.16666667e-02]
 [ 0.00000000e+00 -6.66666667e-01  6.66666667e-01  1.66666667e-01
  -1.66666667e-01]
 [ 1.00000000e+00  2.37904934e-16 -1.25000000e+00 -5.94762335e-17
   2.50000000e-01]
 [-2.29093640e-18  6.66666667e-01  6.66666667e-01 -1.66666667e-01
  -1.66666667e-01]
 [ 1.14546820e-18 -8.33333333e-02 -4.16666667e-02  8.33333333e-02
   4.16666667e-02]]


If I multiply the inverse result with the original matrix, the result is almost the identity matrix but not quite.

In [5]:
print(intMinv @ intM)

[[ 1.00000000e+00  2.08166817e-17 -1.00228468e-18  4.16333634e-17
   1.11022302e-16]
 [-8.88178420e-16  1.00000000e+00  0.00000000e+00 -3.88578059e-16
  -8.88178420e-16]
 [ 8.88178420e-16  2.22044605e-16  1.00000000e+00  5.55111512e-16
   8.88178420e-16]
 [ 0.00000000e+00  0.00000000e+00 -2.29093640e-18  1.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.14546820e-18 -6.93889390e-18
   1.00000000e+00]]


How can we do operations like this, but keep the results in the rational numbers? In this work, I aimed to define matrix operations purely using rational numbers.

## Using the package: RationalAlgebra

The matrix is passed to `RationalMatrix()` function, which instantiates it as a matrix of rational numbers.

In [6]:
import RationalAlgebra.RationalAlgebra as ra

In [7]:
rationalM = ra.RationalMatrix(intM)
print(rationalM)

[[ 1,  1,  1,  1,  1],
 [-2, -1,  0,  1,  2],
 [ 4,  1,  0,  1,  4],
 [-8, -1,  0,  1,  8],
 [16,  1,  0,  1, 16]]


Then, functions such as `inv()` can be used to perform operations. The result is a matrix of rational numbers!

In [8]:
Minv = ra.inv(rationalM)
print(Minv)

[[    0,  1/12, -1/24, -1/12,  1/24],
 [    0,  -2/3,   2/3,   1/6,  -1/6],
 [    1,     0,  -5/4,     0,   1/4],
 [    0,   2/3,   2/3,  -1/6,  -1/6],
 [    0, -1/12, -1/24,  1/12,  1/24]]


We can verify that the product of the matrix with its inverse is exactly the identity matrix.

In [9]:
print(Minv @ rationalM)

[[1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1]]


## Other features: rational vectors

We can also instatiate row and column vectors of rational numbers.

In [10]:
from math import factorial
def vectorMultiplicand(d, N):
    C = np.zeros((N, 1))
    C = C.astype(int)
    C[d] = factorial(d)
    return C

In [11]:
intC = vectorMultiplicand(3,5)
rationalC = ra.RationalVector(intC)
print(rationalC)

[[0],
 [0],
 [0],
 [6],
 [0]]


We can then perform multiplication between matricies and vectors (as well as other combinations).

In [12]:
print( ra.inv(rationalM) @ rationalC )

[[-1/2],
 [   1],
 [   0],
 [  -1],
 [ 1/2]]


## Other features: LU decomposition

The rational inverse algorithm is implemented by a [LUP decomposition](https://en.wikipedia.org/wiki/LU_decomposition), with partial piviting. The $L, U, P$ matricies can be called with the `lu()` function. 

In [13]:
L, U, P = ra.lu(rationalM)
print(L)
print(U)

[[     1,      0,      0,      0,      0],
 [  1/16,      1,      0,      0,      0],
 [  -1/8, -14/15,      1,      0,      0],
 [   1/4,    4/5,   -6/7,      1,      0],
 [  -1/2,  -8/15,    4/7,    1/2,      1]]
[[   16,     1,     0,     1,    16],
 [    0, 15/16,     1, 15/16,     0],
 [    0,     0, 14/15,     2,     4],
 [    0,     0,     0,  12/7,  24/7],
 [    0,     0,     0,     0,    12]]


We can verify that the decomposition worked by checking if $PM = LU$.

In [14]:
print( L @ U )
print( P @ rationalM )

[[16,  1,  0,  1, 16],
 [ 1,  1,  1,  1,  1],
 [-2, -1,  0,  1,  2],
 [ 4,  1,  0,  1,  4],
 [-8, -1,  0,  1,  8]]
[[16,  1,  0,  1, 16],
 [ 1,  1,  1,  1,  1],
 [-2, -1,  0,  1,  2],
 [ 4,  1,  0,  1,  4],
 [-8, -1,  0,  1,  8]]


## Testing

Testing is provided by the `test_basic.py` script.

In [15]:
!python tests/test_basic.py

................................
----------------------------------------------------------------------
Ran 32 tests in 0.034s

OK


## Future work

This was a small project, but in the future I want to add more features and operations.

To-do:
- Implement non-square matricies
- Implement getters and setters
- Implement convenience functions
    - Transpose
    - Matrix of zeros/ones